## Data Loader

This notebook integrates a set of classes and functions for working with recipes and ingredients.


Necessary Imports

In [ ]:
# * -----------------------------Necessary Imports ----------------------------- #
import json
import pandas as pd

In [1]:

# * ----------------------------- CLASS DATALOADER ----------------------------- #
class DataLoader:
    """Clase genérica para procesar información de un archivo csv o json."""

    def __init__(self, file_path, file_type="json"):
        self._file_path = file_path
        self._file_type = file_type

    def load_data(self):
        if self._file_type == "json":
            with open(self._file_path, "r") as f:
                data = json.load(f)
            df = pd.DataFrame(data)
        elif self._file_type == "csv":
            df = pd.read_csv(self._file_path)
        else:
            raise ValueError(f"Unsupported file type: {self._file_type}")
        
        return df

### Create Data Frames for each csv files.

The DataLoader instance loads four different DataFrames from csv files:

- `df_recipe`: contains information about different recipe IDs, recipe names, and other recipe details.
- `df_ingredients`: contains information about ingredient names, synonyms, and entity IDs.
- `df_compound_ingredients`: contains information about compound ingredient names, synonyms, and constituent ingredients.
- `df_relation_recipe_ingredients`: contains information about the relationship between recipe IDs and ingredient names, as well as the original ingredient names and entity IDs.

In [2]:
df_recipe = DataLoader("database/01_Recipe_Details.csv","csv").load_data()
df_ingredients = DataLoader("database/02_Ingredients.csv","csv").load_data()
df_compound_ingredients = DataLoader("database/03_Compound_Ingredients.csv", "csv").load_data()
df_relation_recipe_ingredients = DataLoader("database/04_Recipe-Ingredients_Aliases.csv", "csv").load_data()

#Renaming the Colums 
df_recipe = df_recipe.rename(columns={"Recipe ID": "Recipe_ID"})
df_ingredients = df_ingredients.rename(columns={"Aliased Ingredient Name": "Ingredient_Name", "Ingredient Synonyms": "Synonyms", "Entity ID": "Entity_ID"})
df_compound_ingredients = df_compound_ingredients.rename(columns={"Compound Ingredient Name": "Compound_Name","Compound Ingredient Synonyms": "Compound_Synonyms","Contituent Ingredients": "Contituent"})
df_relation_recipe_ingredients = df_relation_recipe_ingredients.rename(columns={"Recipe ID": "Recipe_ID","Original Ingredient Name": "Original_Name", "Aliased Ingredient Name": "Ingredient_Name", "Entity ID": "Entity_ID" })


### Data Cleaning

Before we can start working with the data, we need to clean it up. First, we'll rename some of the columns to make them easier to work with

In [ ]:
def add_ingredients_to_recipe(df_recipe,df_relation_recipe_ingredients):
    # Agrupar los ingredientes por receta
    ingredientes_por_receta = df_relation_recipe_ingredients.groupby("Recipe_ID")["Ingredient_Name"].apply(list)
    # Crear una nueva columna en el dataframe de recetas con los ingredientes
    df_recipe["Ingredients"] = df_recipe["Recipe_ID"].apply(lambda x: ingredientes_por_receta.get(x, []))
    return df_recipe

df_recipe= add_ingredients_to_recipe(df_recipe,df_relation_recipe_ingredients)
print(df_recipe)

## Pre-processing

In [8]:
df_recipe[df_recipe.Recipe_ID >200]


,Recipe_ID,Title,Source,Cuisine,Ingredients
200,201,bendhi masala,TARLA_DALAL,Indian Subcontinent,"[bean , cayenne , coriander , curry leaf , gin..."
201,202,bengali baingan bhurta,TARLA_DALAL,Indian Subcontinent,"[coriander , egg , garam masala , ghee , sunfl..."
202,203,bengali paneer laddoo,TARLA_DALAL,Indian Subcontinent,"[cardamom , cheese cottage , ghee , milk , sun..."
203,204,bengali style okra,TARLA_DALAL,Indian Subcontinent,"[cayenne , mustard oil , okra , poppy seed , s..."
204,205,bengali tomato chutney,TARLA_DALAL,Indian Subcontinent,"[ginger , sunflower ]"
...,...,...,...,...,...
45767,45768,Strawberries & Cream Frappé,EPICURIOUS,USA,"[half half , ice , strawberry , syrup ]"
45768,45769,Chocolate Whiskey Bundt Cake,EPICURIOUS,USA,"[baking soda , butter , cocoa powder , coffee ..."
45769,45770,"Wild Rice with Butternut Squash, Leeks, and Corn",EPICURIOUS,USA,"[butter , butternut squash , corn , leek , oli..."
45770,45771,Fruit Crumble,EPICURIOUS,USA,"[almond , butter , flour , ice cream , nectari..."
